In [1]:
import os
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import time
import os
from dotenv import load_dotenv

In [2]:
## Función

def apendizador_leads(ruta,sector):
    dfs = []

    for entry in os.scandir(ruta):
        if entry.is_file() and sector in entry.name:
            df = pd.read_csv(entry.path)

            # -------------------------
            # LIMPIEZA DE DATOS (ejemplo)
            # -------------------------
            df = df.dropna(subset=['Email'])          # elimina registros sin email
        

            # Guarda el dataframe limpio
            dfs.append(df)

    # Unir todos los dataframes en uno solo
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_excel(ruta + f"/leads_{sector}_completo.xlsx", index=False)

    return df_final

In [17]:
df = apendizador_leads(r"C:\Users\ejgar\OneDrive\Documentos\GitHub\Leads_sheacher\data\7.Spa","Spa")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nombre     111 non-null    object
 1   Direccion  111 non-null    object
 2   Telefono   110 non-null    object
 3   Web        111 non-null    object
 4   Email      111 non-null    object
 5   Sector     111 non-null    object
 6   Ubicacion  111 non-null    object
dtypes: object(7)
memory usage: 6.2+ KB


In [6]:
df.head()

,Nombre,Direccion,Telefono,Web,Email,Sector,Ubicacion
0,JG fitness Alcobendas | Entrenador personal & ...,"P.º de la Chopera, 182, 28100 Alcobendas, Madrid",+34 604 97 78 56,https://jgfitness.es/centros/alcobendas/,DSC03954_@sharonlopez-1.webp,Entrenador Personal,Alcobendas
1,Nkon Training | Centro de Entrenamiento Person...,"Av. de la Industria, 34, Primera Planta. Local...",NaN,https://www.nkontraining.es/,info@nkontraining.es,Entrenador Personal,Alcobendas
2,SANO Alcobendas - Entrenamiento Personal y Gru...,"Av. de Camilo José Cela, 34, 28108 Alcobendas,...",+34 614 14 27 03,https://www.sanocenter.es/empresas/sano-alcobe...,sanomadridalcobendas@sanocenter.es,Entrenador Personal,Alcobendas
3,ENTRENADOR PERSONAL (PAUL CORDONES) Moraleja -...,"Pl. de la Moraleja, 28109 Alcobendas, Madrid",+34 653 04 36 43,http://paulcordones.com/,info@paulcordones.com,Entrenador Personal,Alcobendas
4,Jorge Morillas Personal Trainer,"C. de la Libertad, 49, 28100 Alcobendas, Madrid",+34 689 48 86 33,https://jorgemorillas.com/,jorgemorillas.trainer@gmail.com,Entrenador Personal,Alcobendas


## Envío de mail

In [3]:
df = pd.read_excel(r"C:\Users\ejgar\OneDrive\Documentos\GitHub\Leads_sheacher\data\7.Spa\leads_Spa_completo.xlsx")

In [ ]:
# Credenciales Gmail
load_dotenv()

EMAIL_REMITENTE = os.getenv("EMAIL_REMITENTE")
PASSWORD_APP = os.getenv("PASSWORD_APP")
SMTP_SERVER = os.getenv("SMTP_SERVER")
SMTP_PORT = int(os.getenv("SMTP_PORT"))

EMAIL_REMITENTE = EMAIL_REMITENTE
PASSWORD_APP = PASSWORD_APP

# servidor SMTP de Gmail
SMTP_SERVER = SMTP_SERVER
SMTP_PORT = SMTP_PORT

# conexión
server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
server.starttls()
server.login(EMAIL_REMITENTE, PASSWORD_APP)

#Contador de mails enviaados
contador = 0

for _, row in df.iterrows():
    ### Pruebas ###
    # nombre = "Enrique" #Para pruebas
    # destinatario = "muzquiz.10@gmail.com"  #Para pruebas
    ## Fin Pruebas ##

    destinatario = row['Email']
    nombre = row['Nombre']
    
    mensaje = MIMEMultipart()
    mensaje['From'] = EMAIL_REMITENTE
    mensaje['To'] = destinatario
    mensaje['Subject'] = "Datos claros y una presencia online profesional para tu negocio"

    cuerpo = f"""
Hola {nombre},

Soy Enrique García, consultor freelance y fundador de EGM Analytics.

Ayudo a pequeños negocios y profesionales en dos áreas clave:

Business Intelligence y análisis de datos: organización y modelado de datos, automatización de informes, análisis para la toma de decisiones y mejora de procesos basados en datos.

Desarrollo web profesional: creación de páginas web claras y bien estructuradas que ayuden a tener presencia online, transmitir credibilidad y mostrar el negocio de forma profesional.

En muchos casos me encuentro con empresas que tienen datos dispersos o infrautilizados y, al mismo tiempo, una presencia digital que no refleja realmente lo que hacen o el valor que aportan.

Mi enfoque es práctico y cercano: soluciones sencillas, bien pensadas y adaptadas a cada negocio, sin complicaciones innecesarias.

Si te parece, podemos agendar una llamada breve (15–20 minutos, sin compromiso) para comentar tu situación y ver si puedo ayudarte en alguno de estos ámbitos.

También me puedes conocer a través de mi web: https://egmanalytics.com

Un saludo,


Enrique García
Consultor en Data, BI y Desarrollo Web
EGM Analytics
https://egmanalytics.com
"""
    mensaje.attach(MIMEText(cuerpo, 'plain'))

    try:
        server.send_message(mensaje)
        print(f"✔ Email enviado a {destinatario}")
        contador += 1
        print(f"Emails enviados hasta ahora: {contador}")
        time.sleep(5)
    except Exception as e:
        print(f"❌ Error enviando a {destinatario}: {e}")
        time.sleep(15)

server.quit()


✔ Email enviado a john@doe.com
Emails enviados hasta ahora: 1
✔ Email enviado a info@privadospamadrid.es
Emails enviados hasta ahora: 2
✔ Email enviado a info@kamaispa.com
Emails enviados hasta ahora: 3
✔ Email enviado a info@amazonia-group.com
Emails enviados hasta ahora: 4
✔ Email enviado a reservas@hotelamura.com
Emails enviados hasta ahora: 5
✔ Email enviado a info@flotexperience.es
Emails enviados hasta ahora: 6
✔ Email enviado a mrmad-spa@mohg.com
Emails enviados hasta ahora: 7
✔ Email enviado a clientes@go-fit.es
Emails enviados hasta ahora: 8
✔ Email enviado a info@soldevillaestetica.es
Emails enviados hasta ahora: 9
✔ Email enviado a esteticamakeuplr@gmail.com
Emails enviados hasta ahora: 10
✔ Email enviado a wolpertesteticamadrid@gmail.com
Emails enviados hasta ahora: 11
✔ Email enviado a recepcion.lm@zagrossports.com
Emails enviados hasta ahora: 12
✔ Email enviado a info@esteticaeleden.com
Emails enviados hasta ahora: 13
✔ Email enviado a info@armonybmc.es
Emails enviados ha

(221,
 b'2.0.0 closing connection 5b1f17b1804b1-47ec5d95edbsm149544235e9.3 - gsmtp')